# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [62]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [63]:
cities = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,husavik,66.0449,-17.3389,49.91,98.0,100.0,11.03,IS,1.627490e+09
1,kapaa,22.0752,-159.3190,77.32,80.0,40.0,10.36,US,1.627490e+09
2,mbaiki,3.8678,17.9892,77.50,68.0,100.0,1.66,CF,1.627490e+09
3,crab hill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,yellowknife,62.4560,-114.3525,62.33,72.0,75.0,9.22,CA,1.627490e+09
...,...,...,...,...,...,...,...,...,...
597,huancavelica,-12.7667,-74.9833,44.67,51.0,92.0,8.21,PE,1.627490e+09
598,moindou,-21.6924,165.6770,58.95,69.0,25.0,5.97,NC,1.627490e+09
599,usinsk,65.9939,57.5281,46.27,92.0,100.0,8.61,RU,1.627490e+09
600,abu kamal,34.4506,40.9171,100.31,15.0,0.0,15.46,SY,1.627490e+09


In [64]:
cities = cities.dropna()
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,husavik,66.0449,-17.3389,49.91,98.0,100.0,11.03,IS,1.627490e+09
1,kapaa,22.0752,-159.3190,77.32,80.0,40.0,10.36,US,1.627490e+09
2,mbaiki,3.8678,17.9892,77.50,68.0,100.0,1.66,CF,1.627490e+09
4,yellowknife,62.4560,-114.3525,62.33,72.0,75.0,9.22,CA,1.627490e+09
5,severo-kurilsk,50.6789,156.1250,56.16,95.0,100.0,7.85,RU,1.627490e+09
...,...,...,...,...,...,...,...,...,...
597,huancavelica,-12.7667,-74.9833,44.67,51.0,92.0,8.21,PE,1.627490e+09
598,moindou,-21.6924,165.6770,58.95,69.0,25.0,5.97,NC,1.627490e+09
599,usinsk,65.9939,57.5281,46.27,92.0,100.0,8.61,RU,1.627490e+09
600,abu kamal,34.4506,40.9171,100.31,15.0,0.0,15.46,SY,1.627490e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [65]:
humidity = cities["Humidity"]
max_humidity = humidity.max()

In [66]:
locations = cities[["Lat","Lng"]]

###  Please note the Humidity Heat Map screenshot was placed in the Images folder \Python-API-challenge\Images

In [67]:
# Plot Heatmap
fig = gmaps.figure()

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [69]:
ideal_cities = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) 
                          & (cities["Wind Speed"] < 10) & (cities["Cloudiness"] == 0)]
ideal_cities.head()                                                                                                                

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,besskorbnaya,44.6447,41.3110,70.50,60.0,0.0,9.66,RU,1.627490e+09
216,fukue,32.6881,128.8419,78.80,81.0,0.0,4.92,JP,1.627490e+09
250,saint-francois,46.4154,3.9054,75.47,59.0,0.0,6.91,FR,1.627490e+09
275,dzaoudzi,-12.7887,45.2699,75.09,73.0,0.0,6.91,YT,1.627490e+09
295,goderich,43.7501,-81.7165,79.03,78.0,0.0,6.91,CA,1.627490e+09


In [70]:
ideal_cities = ideal_cities.dropna()
ideal_cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,besskorbnaya,44.6447,41.3110,70.50,60.0,0.0,9.66,RU,1.627490e+09
216,fukue,32.6881,128.8419,78.80,81.0,0.0,4.92,JP,1.627490e+09
250,saint-francois,46.4154,3.9054,75.47,59.0,0.0,6.91,FR,1.627490e+09
275,dzaoudzi,-12.7887,45.2699,75.09,73.0,0.0,6.91,YT,1.627490e+09
295,goderich,43.7501,-81.7165,79.03,78.0,0.0,6.91,CA,1.627490e+09


In [71]:
ideal_cities.reset_index(inplace=True)
ideal_cities.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,81,besskorbnaya,44.6447,41.3110,70.50,60.0,0.0,9.66,RU,1.627490e+09
1,216,fukue,32.6881,128.8419,78.80,81.0,0.0,4.92,JP,1.627490e+09
2,250,saint-francois,46.4154,3.9054,75.47,59.0,0.0,6.91,FR,1.627490e+09
3,275,dzaoudzi,-12.7887,45.2699,75.09,73.0,0.0,6.91,YT,1.627490e+09
4,295,goderich,43.7501,-81.7165,79.03,78.0,0.0,6.91,CA,1.627490e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [72]:
hotel_list = []

In [73]:
len(ideal_cities)

6

In [74]:
for index, row in ideal_cities.iterrows():
    lat = ideal_cities.loc[index]["Lat"]
    lng = ideal_cities.loc[index]["Lng"]
    
    # Use nearby search with a radius of 5000 for hotels
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
          "location": f'{lat},{lng}',
          "radius": 5000,
          "types": "hotel",
          "key": g_key
         }  
    
    requested = requests.get(base_url, params=params)
    hotel_jsn = requested.json()
    print(json.dumps(hotel_jsn, indent=4, sort_keys=True))
    try:
          hotel_list.append(hotel_jsn['results'][0]['name'])
    except:
          hotel_list.append("")
                
ideal_cities["Hotel Name"] = hotel_list
ideal_cities = ideal_cities.dropna()
ideal_cities.head()

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAnKXqQT71Rk3Wf50MOzf5Mcj0XnFqPxtQFJxBGNtppnx7k6zIB6ajCzeGx22dwj0OpnVPUAOCIvyIuLZPooma-SkCrfRf5SlFG2nc1-dl6uQae1gqSjToRM3jHqe1HdsmUnFRVykKfDDw0SgJhjRRpammSyRk4uitpibSuq1r1Gl9vSyS4NxUj9R4_9bK-oq0v_SVwweoCnyZtIk7WpjN3UlLGM3zwliktgSe6l0h1GsIhQFxTUEh29wlfohoRakgeEte49YEqV56CBVGpm8f9ppRKLvpgan2KSCFY51zmLX3dHZKR1_oKky8KsQ5SqzNjgqT42JeV2fMci5yyyiG3oQ-C2YgAAD5ceDA0xPq0ayPaQoqdRpScX7LUudAk_UO1HjydFdujPTYLgyLEyLXQGnjc4cqkJTdsZbKt3f5A12aY69tIcw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 44.6442379,
                    "lng": 41.3033701
                },
                "viewport": {
                    "northeast": {
                        "lat": 44.67727998201688,
                        "lng": 41.3481629749635
                    },
                    "southwest": {
                        "lat": 44.61233894099409,
                        "lng": 41.27431904511095
 

{
    "html_attributions": [],
    "next_page_token": "Aap_uEA097lp8tIlAaCaC6WozDYicHOJk7CxDUHPPM14VWM8x9JZ6htg-sKM5Wj6jAxybiV32gSwe6YusdnMooWbu0atgUT23Fr4ox_8RFYd2sGZoLdREqlxWTCoBkF7M3OgZZXebZOSmUAdzJehn0rQKEhttw2rAU91NYxNUKhNmCYlkTMrfByN3FY3aFEdmfMbMVGyOyqfRQZxgcwOcMSGhQVGXclOgRTADgurOxSuvr1xvSRXoR3mw07XgjVjJ91BbGdLIAW8jHsMq84au6jLgZ3tM9CEZSDGA64iNyh8t6yRYeDcdhwzWhH9TDSirzTLF1qrttSxQtzUewHMFO_1kSKdne_ZQU7AmsdBvCWG84RHDwfVALAgoDcDH70CXt8yl9VWfUMDELjkoNJy9_67XWFGJRqx-WcI6bSVHWzo_IWOzVlgQ8P2dw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 32.6954809,
                    "lng": 128.8409418
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.89693994072313,
                        "lng": 129.007634028463
                    },
                    "southwest": {
                        "lat": 31.96706300687531,
                        "lng": 128.104468995414
 

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAAcCfggVvHHJYvL-Rkj8fE6ZSBujmqzlgaaCDxJxgyCvM-IRo_9KLV_BV5lC-q3DE1KgjatLGgv_SKVesHkFycKG4OP9gdIPNPsAei9ae3woUfaJ4sxA5pf1WkaS0lGAhxQcZTJeBC7hMR2PEkfB0NZkIvL6d1dGTmNw3enwXGjineAUX17UpbcM5LzGRUAxgPkYZiyayDyvAzgsX8Clre4b18EGiwHXNxeiW9cjlTpO5_CLQFHWFdGj5swuuLfUsC9KxpNDuxB6jOdYGfXVDUPmWx0hw_IzT4GLLpKlrnITvnCspURmllNZ_PwbkcUeMYhO_jmQmJEeoS51gnTP0IHobbyGKzaD9OrjGX1BD-nFO4jgB9e0JGcs9UGshOOxBUL0M3yRHGumm7xcgB_Up2cGE72C5B4JgrSY7sA_c9grj7AUlRIw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 46.465961,
                    "lng": 3.93625
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.49627000824559,
                        "lng": 3.966341055395057
                    },
                    "southwest": {
                        "lat": 46.42641204729319,
                        "lng": 3.88709897734478
     

{
    "html_attributions": [],
    "next_page_token": "Aap_uEC4tIsXyjpf_wPqfv30FDET-Uzkb25kJtlkjhGFUbp8DIW2hCv6y2hDeV0oakjrYUhSwBM7qoUDvN3MHHnhGdqRlMsOktM6tbmmaWDsXj8w521BGTI0Rbq6PykJtNbNKTxVQSFQpB8_L9IMeeaKJAKFx9TZT4WnJg6m1GyMzAtfBMFY8XW2bTXQGZW6vx113KWlvYnHraVOnUvd0sPcdjU9Sa_Ih8-LqpZE4OFDXxQF1Pi1QqPsxBdj8vR90Ll7JWarK_Yg_tOV28qr-cPykA7f025OBs_MpdpSX9Ac-Zj85KhRjt_RjEny4ysG_lvm2We405rTIrrDYwk5Ch7xTLJrwBdFi6CHYRqRbxOU9IsuY2KFjXXYC_R7xuB9vzMQ85Mt293tN_xq0L8IJdTGY6Gdzc_S9divkk3v_mX85bLWjBsVvUDSeA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -12.7809488,
                    "lng": 45.227872
                },
                "viewport": {
                    "northeast": {
                        "lat": -12.77483235306049,
                        "lng": 45.23526681022716
                    },
                    "southwest": {
                        "lat": -12.78627649009456,
                        "lng": 45.2192592380378

{
    "html_attributions": [],
    "next_page_token": "Aap_uECXK1GH-DBP9bsuyc-X-wfsT2syXcO4YIhwIF7ztKC9ggBWe5RrI3OlWw0-LpWWgax5NR4fI4ZZbJu6RvkVzLRgD5ePTw_EaxVogL_KdMm73xgHD8vzvOr2KNBnmV5jz3Uf7g4wK9RYmkbft2FqUFhTKw9Kz0ZRywT4USG37g-RGQZANvoM0_-AUVDuftUgxZbOjdhaOenCgeYbIOeWr5E6aYeWw8dNbiG2q6Gt4LhgKGXXZ-Y8zoyoUKa-2v4dHOry1UJpRZgfZNzE_tIUW7qAUJflqn9o5K0Q6uovzugvKtr2umqGReXFVhZCiEIxptusXaV8PW34gAQo7MuaNsxIR7DqkjQSHnEe664aJsMsR2ZJPPFqJrDc3GRu3DuNIRPmCwQYuAi2pSWEwgcq8byNa0y1JLKRdyxt7TPIaIISIXN8Y2KlMw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 43.7428345,
                    "lng": -81.7138942
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.75457494402696,
                        "lng": -81.6850349384007
                    },
                    "southwest": {
                        "lat": 43.72113901479754,
                        "lng": -81.7350439338909

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 61.2776586,
                    "lng": 108.0136671
                },
                "viewport": {
                    "northeast": {
                        "lat": 61.28053719782526,
                        "lng": 108.0188667255404
                    },
                    "southwest": {
                        "lat": 61.27401436015011,
                        "lng": 108.0070208245359
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png",
            "icon_background_color": "#7B9EB0",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet",
            "name": "Yerbogachen",
            "photos": [
                {
                    "height": 2322,
                    "html_attributions": [

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,81,besskorbnaya,44.6447,41.3110,70.50,60.0,0.0,9.66,RU,1.627490e+09,Besskorbnaya
1,216,fukue,32.6881,128.8419,78.80,81.0,0.0,4.92,JP,1.627490e+09,Goto
2,250,saint-francois,46.4154,3.9054,75.47,59.0,0.0,6.91,FR,1.627490e+09,Molinet
3,275,dzaoudzi,-12.7887,45.2699,75.09,73.0,0.0,6.91,YT,1.627490e+09,Mamoudzou
4,295,goderich,43.7501,-81.7165,79.03,78.0,0.0,6.91,CA,1.627490e+09,Goderich


In [75]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]


###  Please note the Hotel Map screenshot was placed in the Images folder \Python-API-challenge\Images

In [76]:
# Add marker layer ontop of heat map
# markers = gmaps.marker_layer(locations)
locations = ideal_cities[["Lat", "Lng"]]
fig = gmaps.figure(center=(31.0, 31.0), zoom_level=1.7)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

#fig = gmaps.figure()
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))